# Forest Fire Simulation

In this project, I will simulate the spread of a forest fire on a 100×100 grid. Each cell in the grid can be in one of four states: tree, grassland, burning tree, or burnt tree. Based on the simulation I will determine how forest density influences fire spread and I will explores the effect of wind.

The entire notebook should run in about 1 min.

## Part 1: Initial Setup of the Grid
- A 100×100 NumPy array represents the forest.
- Each cell can have one of the following values:
	0: Grassland (non-flammable)
	1: Tree
	2: Burning tree
	3: Burnt tree
- Random Initialization: 
	- A function initializes the grid with trees and grassland based on a tree density parameter (between 0 and 1).
	- Each cell should be a tree with probability = density, and grassland otherwise.
- Lightning Strike: A randomly selected cell that contains a tree (1) will be set to burning (2).

## Part 2: Fire Spread Simulation
- Simulation Step:
	- In each time step:
		- A burning tree (2) becomes a burnt tree (3).
		- All trees (1) that are directly adjacent to a burning tree catch fire and become burning trees (2) in the next step.
- Run of the Simulation:
    - The simulation is continued step-by-step until no more trees are burning.
- Results:
    - The absolute and relative number of burnt trees will be considered.


## Part 3: Visualization and Analysis
- Graphical Representation:
    - Creation of an animated gif for a full run of the simulation, where each frame/picture corresponds to a time-step.
- Density Curve:
    - Creation of a plot showing the percentage of trees burnt as a function of the initial density.
    - The critical density (i.e. the density above which more than 90% of trees burn down) will be plotted.
    

## Part 4: Extensions – Wind Effect
- Wind Influence:
    - The effects of wind (from left to right) will be determined for two wind strengths, that influence the spread in different ways:
         - Strength 1: The fire spreads to adjacent cells and, additionally, two cells to the right.
         - Strength 2: The fire spreads one cell up and down as well as three cells to the right, while it does not spread to the left.
    - The observations will be explained.

---

## Part 1

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import imageio.v2 as imageio  # needed for making gifs
import os  # needed to safe and delete frames while making gifs

%matplotlib inline

In [ ]:
def init_forest(tree_density: float, grid=100) -> np.ndarray:
    """
    @param tree_density: probabilty of a cell to contain a tree
    @param grid: size of the grid along one direction, i.e. size = grid x grid
    
    return a forest (100x100 NumPy array) with grass (0) or trees (1), where one random tree burns (2)
    """
    
    # representation of forest grid
    forest = np.random.uniform(size=(grid, grid))
    
    # assign tree- or grass-cells
    # for a uniform distribution with range [0, 1) the probability to lie below "tree_density" is "tree_density"
    tree_mask = (forest < tree_density)
    
    forest[tree_mask] = 1
    forest[~ tree_mask] = 0
    
    # if there are trees: select a random tree (1) to burn (2)
    tree_inds = np.where(forest == 1)  # form: (row indices, column indices)
    tree_number = len(tree_inds[0])
    
    if tree_number != 0:
        burning_ind = np.random.randint(0, tree_number)
    
        row = tree_inds[0][burning_ind]
        column = tree_inds[1][burning_ind]
    
        forest[row, column] = 2
    
    return forest


# test result
colors = ['olive', 'green', 'red']  # for a prettier plot
layout = ListedColormap(colors)
labels = ['Grass', 'Tree', 'Burning']


fig, ax = plt.subplots(figsize=(6,6)) 
plt.rcParams.update({'font.size': 14})

cmap = ax.imshow(init_forest(0.42, 100), origin='lower', cmap=layout)

cbar = fig.colorbar(cmap, ax=ax, ticks=[0, 1, 2], shrink=0.8)
cbar.ax.set_yticklabels(labels)

# remove ticks by fixing them with an empty list
plt.xticks([])
plt.yticks([])

ax.set_title('Forest map', fontsize=16)
plt.show()

## Part 2

In [ ]:
def iter_step(forest: np.ndarray, fire: np.ndarray, directions: tuple, grid=100) -> tuple[np.ndarray, np.ndarray]:
    '''
    @param forest: map of the forest to burn
    @param fire: map of the fire where 0 = 'no fire' and 1 = 'currently or previously burning'
    @param directions: select adjacent cells to burn relative to burning cells, form: ((x1, y1), (x2, y2),...)
    @param grid: size of the grid along one direction, i.e. size = grid x grid
    
    Simulate how fire spreads in one step: cells in specific positions to burning cells start burning (2) if they were trees (1)
    and previously burning cells become burnt (3). Return the forest and fire after the step
    '''
    burning_mask = (forest == 2)
    burning_rows, burning_cols = np.where(burning_mask)  # obtain positions of burning trees
    
    # obtain adjacent cells by adding directions to burning cells
    for direction in directions:
        next_rows = burning_rows + direction[0]
        next_cols = burning_cols + direction[1]
        
        # make sure to not get out of bounds
        range_mask = (0 <= next_rows) & (next_rows < grid) & (0 <= next_cols) & (next_cols < grid)
        
        # extend the fire
        fire[next_rows[range_mask], next_cols[range_mask]] = 1
    
    next_burning_mask = (fire == 1) & (forest == 1)
    forest[next_burning_mask] = 2  # update newly ignited trees
    forest[burning_mask] = 3  # update currently burning trees
    
    return forest, fire    
    

def fire_simulation(tree_density: float, grid=100) -> tuple[float, float]:
    '''
    @param tree_density: probabilty of a cell in the initial grid to contain a tree
    
    take the output of "init_forest", simulate a fire, using "iter_step", and return the relative and absolute number of burnt trees
    '''
    forest = init_forest(tree_density, grid)
    total_trees = np.sum(forest) - 1  # subtract 1 to account for the burning tree

    fire = np.zeros((grid, grid))  # represents fire on the grid: 0 = no fire, 1 = currently or previously burning
    directions = ((1, 0), (-1, 0), (0, 1), (0, -1))  # to later select adjacent files: (up, down, right, left)

    while 2 in forest:
        forest, fire = iter_step(forest, fire, directions, grid)

    # extract results
    burnt_mask = (forest == 3)
    burnt_trees = int(np.sum(burnt_mask))
    percentage_burnt = 100 * burnt_trees / total_trees
    
    return percentage_burnt, burnt_trees
    

# test results for the chosen example
tree_density = 0.6

percentage_burnt, burnt_trees = fire_simulation(tree_density)

print(f'For a forest density of {tree_density}, {burnt_trees} trees were burnt.')
print(f'This corresponds to {percentage_burnt:.2g}% of the initial tree population.')

## Part 3

**Graphical representation**

In [ ]:
colors = ['olive', 'green', 'red', 'black']  # for a prettier plot
layout = ListedColormap(colors)
labels = ['Grass', 'Tree', 'Burning', 'burnt']

# define a function based on the results from part 2
def visualise_grid(tree_density: float) -> list:
    '''
    @param tree_density: probabilty of a cell in the initial grid to contain a tree
    
    simulate a forest fire for a given tree density, save frames of the forest map into the folder 'frames' and return the frames' filenames
    '''
    forest = init_forest(tree_density)
    total_trees = np.sum(forest) - 1
    
    fire = np.zeros((100, 100))  # for iteration steps
    directions = ((1, 0), (-1, 0), (0, 1), (0, -1))  # to later select adjacent files: (up, down, right, left)
    
    # used when plots are safed
    frame = 0
    filenames = []
    
    while 2 in forest:
        forest, fire = iter_step(forest, fire, directions)        
        
        # save current forest map
        fig, ax = plt.subplots(figsize=(6,6)) 
        
        cmap = ax.imshow(forest, origin='lower', cmap=layout)
        cbar = fig.colorbar(cmap, ax=ax, ticks=[0, 1, 2, 3], shrink=0.8)
        cbar.ax.set_yticklabels(labels)

        # remove ticks by fixing them with an empty list
        plt.xticks([])
        plt.yticks([])

        ax.set_title('Forest map', fontsize=16)
        
        # save picture
        filename = f"frames/frame_{frame:03d}.png"
        plt.savefig(filename)
        
        plt.close()
        
        filenames.append(filename)
        frame += 1

    return filenames

        
def make_gif(tree_density: float) -> None:
    '''
    @param tree_density: probabilty of a cell in the initial grid to contain a tree

    create a gif with a simulated forest fire based on the tree density of the forest
    '''
    # Output directory for frames
    os.makedirs("frames", exist_ok=True)

    # simulate fire
    filenames = visualise_grid(tree_density)
        
    # Create GIF
    with imageio.get_writer('forest_map.gif', mode='I', duration=0.1) as writer:
        for filename in filenames:
            image = imageio.imread(filename)
            writer.append_data(image)

    # Clean up individual frames
    for filename in filenames:
        os.remove(filename)

    print("GIF saved as 'forest_map.gif'")
    

# test resulting gif
make_gif(0.8)

**Density curve**

In [ ]:
# define testcase
densities = np.arange(.1, 1., .04)
repetitions = 40  # for statistical accuarcy

# fill with results later
percentages_mean = np.array(())
percentages_std = np.array(())

# calculate percentages (cannot be done vectorised due to init_forest-function)
percentages = np.zeros(repetitions)

for density in densities:
    for i in range(repetitions):
        percentages[i], _ = fire_simulation(density)
        
    # extract mean and std
    mean = np.mean(percentages)
    std = np.std(percentages) / np.sqrt(repetitions)
    
    percentages_mean = np.append(percentages_mean, mean)
    percentages_std = np.append(percentages_std, std)

    
# calculate critical density
mask_90 = (percentages_mean > 90)
crit_density = densities[mask_90][0]

# plot results
fig, ax = plt.subplots(figsize=(8,6)) 

plt.errorbar(densities, percentages_mean, percentages_std, fmt='.', label='data')
plt.vlines(crit_density, 0., 100., label='critical density', linestyles='dashed', color='red', alpha=0.5)

ax.set_title('Density Curve', fontsize=16)
plt.xlabel('Tree density',fontsize=16)
plt.ylabel('Percentage of burnt trees',fontsize=16)
plt.legend(loc='upper left')
plt.grid(True, 'both')

plt.show()

## Part 4

In [ ]:
def windy_fire_simulation(tree_density: float, wind_strength: int, make_gif=False) -> float:
    '''
    @param tree_density: probabilty of a cell in the initial grid to contain a tree
    @param wind_strength: strength of the wind (0 or 1 or 2)
    @param make_gif: Do you want to create GIF?
    
    simulate forest fire for given wind strengths and return ratio of burnt trees
    if needed visualise results with a GIF
    '''
    forest = init_forest(tree_density)
    total_trees = np.sum(forest) - 1  # subtract 1 to account for the burning tree

    fire = np.zeros((100, 100))  # for iteration steps
    
    # necessary for creation of GIFs
    frame = 0
    filenames = []
    
    # choose how the fire spreads relative to the burning tree
    if wind_strength == 0:
        directions = ((1, 0), (-1, 0), (0, 1), (0, -1))
        
    elif wind_strength == 1:
        directions = ((1, 0), (-1, 0), (0, 1), (0, -1), (0, 2))
        
    elif wind_strength == 2:
        directions = ((1, 0), (-1, 0), (0, 1), (0, 2), (0, 3))
        
    else:
        print('Wind strength must be 0, 1 or 2')
        return None

    
    # actual simulation
    while 2 in forest:
        forest, fire = iter_step(forest, fire, directions)
        
        # make a gif, if wanted
        if make_gif:
            fig, ax = plt.subplots(figsize=(6,6)) 
        
            cmap = ax.imshow(forest, origin='lower', cmap=layout)
            cbar = fig.colorbar(cmap, ax=ax, ticks=[0, 1, 2, 3], shrink=0.8)
            cbar.ax.set_yticklabels(labels)

            # remove ticks by fixing them with an empty list
            plt.xticks([])
            plt.yticks([])

            ax.set_title(f'Forest map with strength-{wind_strength} wind', fontsize=16)
        
            # save picture
            filename = f"frames/frame_{frame:03d}.png"
            plt.savefig(filename)
        
            plt.close()
        
            filenames.append(filename)
            frame += 1
         
    if make_gif:
        # Output directory for frames
        os.makedirs("frames", exist_ok=True)
        
        # Create GIF
        with imageio.get_writer(f'fire_{wind_strength}_wind.gif', mode='I', duration=0.1) as writer:
            for filename in filenames:
                image = imageio.imread(filename)
                writer.append_data(image)

        # Clean up individual frames
        for filename in filenames:
            os.remove(filename)

        print(f"GIF saved as 'fire_{wind_strength}_wind.gif'")

    # extract results
    burnt_mask = (forest == 3)
    burnt_trees = int(np.sum(burnt_mask))
    percentage_burnt = 100 * burnt_trees / total_trees
    
    return percentage_burnt


# define testcase
densities = np.arange(.1, 1., .04)
repetitions = 20  # for statistical accuarcy

# fill with results later
perc_mean_1 = np.array(())
perc_std_1 = np.array(())
perc_mean_2 = np.array(())
perc_std_2 = np.array(())

# calculate percentages
perc_1 = np.zeros(repetitions)
perc_2 = np.zeros(repetitions)

for density in densities:
    for i in range(repetitions):
        perc_1[i] = windy_fire_simulation(density, 1)
        perc_2[i] = windy_fire_simulation(density, 2)
        
    # extract means and std's
    mean_1 = np.mean(perc_1)
    std_1 = np.std(perc_1) / np.sqrt(repetitions)
    mean_2 = np.mean(perc_2)
    std_2 = np.std(perc_2) / np.sqrt(repetitions)
    
    perc_mean_1 = np.append(perc_mean_1, mean_1)
    perc_std_1 = np.append(perc_std_1, std_1)
    perc_mean_2 = np.append(perc_mean_2, mean_2)
    perc_std_2 = np.append(perc_std_2, std_2)
    
# calculate critical density (not for wind_strength = 2, because the fire does not reach 90%)
mask_90 = (perc_mean_1 > 90)
crit_density_1 = densities[mask_90][0]

# plot results
fig, ax = plt.subplots(figsize=(8,6)) 

plt.errorbar(densities, percentages_mean, percentages_std, fmt='.', label='data (strength 0)', color='green')
plt.vlines(crit_density, 0., 100., label='crit. density (strength 0)', linestyles='dashed', color='green', alpha=0.5)

plt.errorbar(densities, perc_mean_1, perc_std_1, fmt='.', label='data (strength 1)', color='orange')
plt.vlines(crit_density_1, 0., 100., label='crit. density (strength 1)', linestyles='dashed', color='orange', alpha=0.5)

plt.errorbar(densities, perc_mean_2, perc_std_2, fmt='.', label='data (strength 2)', color='red')

ax.set_title('Density Curve for different wind strengths', fontsize=16)
plt.xlabel('Tree density',fontsize=16)
plt.ylabel('Percentage of burnt trees',fontsize=16)
plt.legend(loc='upper left')
plt.grid(True, 'both')

plt.show()

In [ ]:
# test the GIFs generated in the windy-fire-simulation
_ = windy_fire_simulation(.7, 1, make_gif=True)
_ = windy_fire_simulation(.7, 2, make_gif=True)

**Findings and observations**

For wind strengths of 1 this results in the forest reaching its critical density already at about 0.62. This value is lower than for the windless case because at each step one additional tree can potentially be ignited. Both of these strengths yield quite accurate results for very high or very low densities. Only in the vicinity of their critical densities the errorbars become larger. This is because around and below the critical density the spread of fire greatly depends on the fires starting point.
This is an observation that is also valid for wind strengths of 2. Here, the uncertainties begin to rise around densities of 0.5. For higher densities it only rises slowly and never reaches a critical density. This is because even at a density of 1 the fire's spread is determined by its starting point as it cannot reach any cells west of its starting point.